# Import and Install Dependencies

In [1]:
!pip install tensorflow tensorflow-gpu opencv-python sklearn mediapipe matplotlib

You should consider upgrading via the 'c:\users\artem-laptop\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Model
mp_drawing = mp.solutions.drawing_utils # Drawing

In [3]:
def mediapipe_detections(image, model):
    # Color conversion
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    
    # Model stuff
    results = model.process(image)
    
    # Color conversion back
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [6]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read();
        
        # Make detections
        image, results = mediapipe_detections(frame, holistic)
        
        # Draw landmarks
        draw_landmarks(image, results)

        cv2.imshow('OpenCV feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Extract Keypoint Values

In [59]:
def extract_keypoints(results):
    start_keypoint = results.pose_landmarks.landmark[0]
    pose = np.array([[res.x - start_keypoint.x, res.y - start_keypoint.y, res.z - start_keypoint.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)

    face = np.array([[res.x - start_keypoint.x, res.y - start_keypoint.y, res.z - start_keypoint.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)

    lh = np.array([[res.x - start_keypoint.x, res.y - start_keypoint.y, res.z - start_keypoint.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)

    rh = np.array([[res.x - start_keypoint.x, res.y - start_keypoint.y, res.z - start_keypoint.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    return np.concatenate([pose, face, lh, rh])

In [8]:
extract_keypoints(results)

array([0., 0., 0., ..., 0., 0., 0.])

# Setup folders structure

In [8]:
DATA_PATH = os.path.join('MP_data')
actions = np.array(['Я', 'Ты', 'Глухой'])

# collect 30 videos for any action
no_sequences = 30

# videos are going to be 30 frames in length
sequence_length = 30

In [9]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Collect Keypoints Values for Training and Testing

In [16]:
cap = cv2.VideoCapture(1)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
     # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):
                ret, frame = cap.read();

                # Make detections
                image, results = mediapipe_detections(frame, holistic)

                # Draw landmarks
                draw_landmarks(image, results)
                
                # Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    
                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

# Preprocess Data and Create Labels and Features

In [17]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [18]:
label_map = {label:num for num, label in enumerate(actions)}

In [19]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [20]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# Build and Train LSTM Neural Network

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [22]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [40]:
# Build a model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [41]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [25]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 3s 474ms/step - loss: 1.4967 - categorical_accuracy: 0.3294
Epoch 2/2000
3/3 [==============================] - 0s 54ms/step - loss: 1.1076 - categorical_accuracy: 0.3765
Epoch 3/2000
3/3 [==============================] - 0s 50ms/step - loss: 1.1306 - categorical_accuracy: 0.3765
Epoch 4/2000
3/3 [==============================] - 0s 56ms/step - loss: 1.3319 - categorical_accuracy: 0.2706
Epoch 5/2000
3/3 [==============================] - 0s 50ms/step - loss: 1.1118 - categorical_accuracy: 0.3882
Epoch 6/2000
3/3 [==============================] - 0s 51ms/step - loss: 1.0896 - categorical_accuracy: 0.4118
Epoch 7/2000
3/3 [==============================] - 0s 51ms/step - loss: 1.0580 - categorical_accuracy: 0.4706
Epoch 8/2000
3/3 [==============================] - 0s 50ms/step - loss: 0.9885 - categorical_accuracy: 0.5294
Epoch 9/2000
3/3 [==============================] - 0s 50ms/step - loss: 1.0078 - categorical_accuracy: 0.6000


3/3 [==============================] - 0s 50ms/step - loss: 0.1327 - categorical_accuracy: 0.9294
Epoch 75/2000
3/3 [==============================] - 0s 52ms/step - loss: 0.1275 - categorical_accuracy: 0.9529
Epoch 76/2000
3/3 [==============================] - 0s 50ms/step - loss: 0.2582 - categorical_accuracy: 0.9294
Epoch 77/2000
3/3 [==============================] - 0s 50ms/step - loss: 0.4519 - categorical_accuracy: 0.8471
Epoch 78/2000
3/3 [==============================] - 0s 48ms/step - loss: 0.3665 - categorical_accuracy: 0.8353
Epoch 79/2000
3/3 [==============================] - 0s 50ms/step - loss: 0.2696 - categorical_accuracy: 0.8824
Epoch 80/2000
3/3 [==============================] - 0s 49ms/step - loss: 0.1099 - categorical_accuracy: 0.9882
Epoch 81/2000
3/3 [==============================] - 0s 50ms/step - loss: 0.1841 - categorical_accuracy: 0.9294
Epoch 82/2000
3/3 [==============================] - 0s 49ms/step - loss: 0.1383 - categorical_accuracy: 0.9647
Epoch 

3/3 [==============================] - 0s 51ms/step - loss: 4.5779e-05 - categorical_accuracy: 1.0000
Epoch 146/2000
3/3 [==============================] - 0s 52ms/step - loss: 4.4531e-05 - categorical_accuracy: 1.0000
Epoch 147/2000
3/3 [==============================] - 0s 50ms/step - loss: 4.3305e-05 - categorical_accuracy: 1.0000
Epoch 148/2000
3/3 [==============================] - 0s 53ms/step - loss: 4.2032e-05 - categorical_accuracy: 1.0000
Epoch 149/2000
3/3 [==============================] - 0s 50ms/step - loss: 4.0844e-05 - categorical_accuracy: 1.0000
Epoch 150/2000
3/3 [==============================] - 0s 50ms/step - loss: 3.9824e-05 - categorical_accuracy: 1.0000
Epoch 151/2000
3/3 [==============================] - 0s 50ms/step - loss: 3.8697e-05 - categorical_accuracy: 1.0000
Epoch 152/2000
3/3 [==============================] - 0s 50ms/step - loss: 3.7652e-05 - categorical_accuracy: 1.0000
Epoch 153/2000
3/3 [==============================] - 0s 52ms/step - loss: 3.66

3/3 [==============================] - 0s 70ms/step - loss: 8.3974e-06 - categorical_accuracy: 1.0000
Epoch 216/2000
3/3 [==============================] - 0s 55ms/step - loss: 8.2656e-06 - categorical_accuracy: 1.0000
Epoch 217/2000
3/3 [==============================] - 0s 57ms/step - loss: 8.1226e-06 - categorical_accuracy: 1.0000
Epoch 218/2000
3/3 [==============================] - 0s 58ms/step - loss: 8.0146e-06 - categorical_accuracy: 1.0000
Epoch 219/2000
3/3 [==============================] - 0s 52ms/step - loss: 7.8954e-06 - categorical_accuracy: 1.0000
Epoch 220/2000
3/3 [==============================] - 0s 56ms/step - loss: 7.7692e-06 - categorical_accuracy: 1.0000
Epoch 221/2000
3/3 [==============================] - 0s 59ms/step - loss: 7.6219e-06 - categorical_accuracy: 1.0000
Epoch 222/2000
3/3 [==============================] - 0s 55ms/step - loss: 7.5350e-06 - categorical_accuracy: 1.0000
Epoch 223/2000
3/3 [==============================] - 0s 56ms/step - loss: 7.40

KeyboardInterrupt: 

# Make Predictions 

In [26]:
res = model.predict(X_test)

In [38]:
actions[np.argmax(res[0])]

'Ты'

In [37]:
actions[np.argmax(y_test[0])]

'Ты'

# Test in Real Time 

In [39]:
model.save('action.h5')
del model

In [42]:
model.load_weights('action.h5')

In [43]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [78]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.55

cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detections(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.Solution

Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_bas

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutp

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.S

...

Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.Solu

Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution

Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Глухой
<class 'mediapipe.python.solution

Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs

Глухой
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Я
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Ты
<class 'mediapipe.python.solution_base.Solution

In [73]:
threshold

0.5